## Question answering based on the OpenAI cookbook tutorial

OpenAI has a very thorough tutorial on creating a question answering function using embeddings (https://github.com/openai/openai-cookbook/blob/main/examples/Question_answering_using_embeddings.ipynb).
One of the possible solutions was to follow this tutorial.

The steps modified from the tutorial are: 
- Prepare the laws data
- Create embeddings for the data
- Create embedding for the query
- Find most relevant text sections
- Send query with the question and most relevant sections
- Get query answer


## Creating the embeddings file

A little section about what embeddings are.

### Overview of the data

A section about the data. Selle võib ka tõsta üldisemaks.

In [41]:
# imports
from fastparquet import write
from tables import *
import openai  # for generating embeddings
import pandas as pd  # for DataFrames to store article sections and embeddings
import re  # for cutting <ref> links out of Wikipedia articles
import tiktoken  # for counting tokens
import numpy as np
from dotenv import dotenv_values
from scipy import spatial as spatial_2# for calculating vector similarities for search
import faiss  # for vector database
import scipy.spatial.distance as spatial
import time

In [4]:
#To use embeddings you must create an .env file where the content is OPENAI_API_KEY = "your-api-key"
config = dotenv_values(".env")["OPENAI_API_KEY"]
openai.organization = "org-3O7bHGD9SwjHVDuUCNCGACC3"
openai.api_key = config
GPT_MODEL = "gpt-3.5-turbo"  # only matters insofar as it selects which tokenizer to use
EMBEDDING_MODEL = "text-embedding-ada-002"

In [5]:
df = pd.read_csv("legal_acts_estonia.csv", names=['type', 'nr','text','link'])

In [6]:
df

,type,nr,text,link
0,VVS,para1,§ 1.\nVabariigi Valitsuse pädevus\n(1) Vabarii...,https://www.riigiteataja.ee/akt/VVS#para1
1,VVS,para2,§ 2.\nVabariigi Valitsuse asukoht\nVabariigi V...,https://www.riigiteataja.ee/akt/VVS#para2
2,VVS,para3,§ 3.\nVabariigi Valitsuse liikmed\n(1) Vabarii...,https://www.riigiteataja.ee/akt/VVS#para3
3,VVS,para3b1,§ 3.1.\nVabariigi Valitsuse liikme juurdepääs ...,https://www.riigiteataja.ee/akt/VVS#para3b1
4,VVS,para4,§ 4.\nVabariigi Valitsuse liikmete tööülesanne...,https://www.riigiteataja.ee/akt/VVS#para4
...,...,...,...,...
32784,NaN,para7,§ 7.\nKaitsekulu\nKaitsekulu on NATO meetodi j...,https://www.riigiteataja.ee/akt/#para7
32785,NaN,para8,§ 8.\nEesti Haigekassa eelarvepositsioon\nEest...,https://www.riigiteataja.ee/akt/#para8
32786,NaN,para9,§ 9.\nEelarveaasta jooksul riigiteede üleandmi...,https://www.riigiteataja.ee/akt/#para9
32787,NaN,para10,§ 10.\nTapa spordi- ja vabaajakeskuse ehituse ...,https://www.riigiteataja.ee/akt/#para10


### Cleaning and splitting the data

The tutorial suggests that long sections, which have over 1600 tokens should be split down to smaller sections. This makes sure that the token limit per query is not exceeded later.

In [7]:
def num_tokens_from_string(string: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [8]:
df["title"] = df["text"].str.split("\n").str[1]
df["text"] = df["text"].str.split("\n").apply(lambda x: ','.join(x[2:]))
df["text"] = df["text"].str.replace('\n','')
#have to split up to have less than 1600 tokens
df['split_text'] = df['text'].str.split(r'\(\d+\)')
df["text"] = df["text"].str.replace('\n','')
df = df.explode("split_text")
df = df[df["split_text"]!= ""]
df["nr"] = df["nr"].str.replace('para','')

In [9]:
df['token_count'] = df['split_text'].apply(num_tokens_from_string)
df_cut = df[["type","nr","link","title","split_text"]]
over_length = df[df["token_count"]>1600]
#remove the long strings for now
df = df[df["token_count"]<1600]
over_length['split_text_2'] = over_length['split_text'].str.split(r'\(\d+\.\d+\)')
over_length = over_length.explode("split_text_2")
over_length['token_count'] = over_length['split_text_2'].apply(num_tokens_from_string)
#if they still have too many words, most often the paragraphs are lists of definitions, which can be split by list enumeration
#other with a shorter length can be added back to original dataframe
over_length_merge = over_length[over_length["token_count"]<1600]
over_length_merge = over_length_merge.drop(columns=["split_text"]).rename(columns={"split_text_2":"split_text"})
df = df.append(over_length_merge)

C:\Users\annilo\AppData\Local\Temp\ipykernel_9368\3183686348.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(over_length_merge)


In [10]:
df

,type,nr,text,link,title,split_text,token_count
0,VVS,1,(1) Vabariigi Valitsus teostab täidesaatvat ri...,https://www.riigiteataja.ee/akt/VVS#para1,Vabariigi Valitsuse pädevus,Vabariigi Valitsus teostab täidesaatvat riigi...,39
0,VVS,1,(1) Vabariigi Valitsus teostab täidesaatvat ri...,https://www.riigiteataja.ee/akt/VVS#para1,Vabariigi Valitsuse pädevus,Vabariigi Valitsus teostab täidesaatvat riigi...,35
1,VVS,2,"Vabariigi Valitsuse asukoht on Tallinnas.,",https://www.riigiteataja.ee/akt/VVS#para2,Vabariigi Valitsuse asukoht,"Vabariigi Valitsuse asukoht on Tallinnas.,",15
2,VVS,3,(1) Vabariigi Valitsuse liikmed on peaminister...,https://www.riigiteataja.ee/akt/VVS#para3,Vabariigi Valitsuse liikmed,Vabariigi Valitsuse liikmed on peaminister ja...,18
2,VVS,3,(1) Vabariigi Valitsuse liikmed on peaminister...,https://www.riigiteataja.ee/akt/VVS#para3,Vabariigi Valitsuse liikmed,Ministrite pädevuse ministeeriumi juhtimisel ...,98
...,...,...,...,...,...,...,...
31032,ATKEAS,69b3,(1) Energia aktsiisivabastuse loa taotlemiseks...,https://www.riigiteataja.ee/akt/ATKEAS#para69b3,Energia aktsiisivabastuse loa taotlemisel esit...,Intensiivse gaasitarbimisega ettevõtja esitab...,915
31032,ATKEAS,69b3,(1) Energia aktsiisivabastuse loa taotlemiseks...,https://www.riigiteataja.ee/akt/ATKEAS#para69b3,Energia aktsiisivabastuse loa taotlemisel esit...,Käesoleva paragrahvi lõike 1.3 punktis 2 nime...,153
31035,ATKEAS,69b6,(1) Energia aktsiisivabastuse loa omanik on ko...,https://www.riigiteataja.ee/akt/ATKEAS#para69b6,Energia aktsiisivabastuse loa omaniku kohustused,Energia aktsiisivabastuse loa omanik on kohus...,1223
31035,ATKEAS,69b6,(1) Energia aktsiisivabastuse loa omanik on ko...,https://www.riigiteataja.ee/akt/ATKEAS#para69b6,Energia aktsiisivabastuse loa omaniku kohustused,Energia aktsiisivabastuse luba omav või omanu...,678


In [11]:
over_length = over_length[over_length["token_count"]>1600]
over_length["title_2"] = over_length["split_text_2"].str.split(r',\d+[\.\d+]*\)').str[0]
over_length["split_text_3"] = over_length["split_text_2"].str.split(r',\d+[\.\d+]*\)')
over_length = over_length.explode("split_text_3")
over_length = over_length[over_length["title_2"]!=over_length["split_text_3"]]
over_length['token_count'] = over_length['split_text_3'].apply(num_tokens_from_string)
over_length["title"] = over_length["title"]+ '. ' + over_length["title_2"]
over_length = over_length.drop(columns=["split_text","split_text_2","title_2"]).rename(columns={"split_text_3":"split_text"})
df = df.append(over_length)

C:\Users\annilo\AppData\Local\Temp\ipykernel_9368\3167922143.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(over_length)


In [12]:
df.fillna('', inplace=True)
df["concatenated"] = "Seadus "+ df["type"]+" paragrahv "+ df["nr"]+". Pealkiri: "+ df["title"]+ " Sisu: "+ df["split_text"]
df["concatenated"] = df["concatenated"].str.replace(r'\s+', ' ').str.rstrip(",")


C:\Users\annilo\AppData\Local\Temp\ipykernel_9368\792991765.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df["concatenated"] = df["concatenated"].str.replace(r'\s+', ' ').str.rstrip(",")


In [13]:
df

,type,nr,text,link,title,split_text,token_count,concatenated
0,VVS,1,(1) Vabariigi Valitsus teostab täidesaatvat ri...,https://www.riigiteataja.ee/akt/VVS#para1,Vabariigi Valitsuse pädevus,Vabariigi Valitsus teostab täidesaatvat riigi...,39,Seadus VVS paragrahv 1. Pealkiri: Vabariigi Va...
0,VVS,1,(1) Vabariigi Valitsus teostab täidesaatvat ri...,https://www.riigiteataja.ee/akt/VVS#para1,Vabariigi Valitsuse pädevus,Vabariigi Valitsus teostab täidesaatvat riigi...,35,Seadus VVS paragrahv 1. Pealkiri: Vabariigi Va...
1,VVS,2,"Vabariigi Valitsuse asukoht on Tallinnas.,",https://www.riigiteataja.ee/akt/VVS#para2,Vabariigi Valitsuse asukoht,"Vabariigi Valitsuse asukoht on Tallinnas.,",15,Seadus VVS paragrahv 2. Pealkiri: Vabariigi Va...
2,VVS,3,(1) Vabariigi Valitsuse liikmed on peaminister...,https://www.riigiteataja.ee/akt/VVS#para3,Vabariigi Valitsuse liikmed,Vabariigi Valitsuse liikmed on peaminister ja...,18,Seadus VVS paragrahv 3. Pealkiri: Vabariigi Va...
2,VVS,3,(1) Vabariigi Valitsuse liikmed on peaminister...,https://www.riigiteataja.ee/akt/VVS#para3,Vabariigi Valitsuse liikmed,Ministrite pädevuse ministeeriumi juhtimisel ...,98,Seadus VVS paragrahv 3. Pealkiri: Vabariigi Va...
...,...,...,...,...,...,...,...,...
30972,ATKEAS,36,(1) Aktsiisilaopidaja ja registreeritud kaubas...,https://www.riigiteataja.ee/akt/ATKEAS#para36,Aktsiisilaopidaja ja registreeritud kaubasaaja...,teatama maksuhaldurile kütuse erimärgistamise...,57,Seadus ATKEAS paragrahv 36. Pealkiri: Aktsiisi...
30972,ATKEAS,36,(1) Aktsiisilaopidaja ja registreeritud kaubas...,https://www.riigiteataja.ee/akt/ATKEAS#para36,Aktsiisilaopidaja ja registreeritud kaubasaaja...,esitama maksuhaldurile SADHESi lisadokumentid...,131,Seadus ATKEAS paragrahv 36. Pealkiri: Aktsiisi...
30972,ATKEAS,36,(1) Aktsiisilaopidaja ja registreeritud kaubas...,https://www.riigiteataja.ee/akt/ATKEAS#para36,Aktsiisilaopidaja ja registreeritud kaubasaaja...,pidama aktsiisilaost lähetatud denatureeritud...,80,Seadus ATKEAS paragrahv 36. Pealkiri: Aktsiisi...
30972,ATKEAS,36,(1) Aktsiisilaopidaja ja registreeritud kaubas...,https://www.riigiteataja.ee/akt/ATKEAS#para36,Aktsiisilaopidaja ja registreeritud kaubasaaja...,"[kehtetu - RT I, 03.04.2018, 2 - jõust. 01.02...",31,Seadus ATKEAS paragrahv 36. Pealkiri: Aktsiisi...


In [14]:
#replace all list enumeration
#dont think it is needed
#df["split_text"] = df["split_text"].str.replace(r',\d+[\.\d+]*\)','')

In [15]:
laws = np.array(df["concatenated"])
#laws = laws[:10000]

In [16]:
# calculate embeddings

##
##DO NOT RUN UNLESS NEED TO CREATE NEW EMBEDDINGS (THIS CODE COSTS ABT 2 DOLLARS)

#EMBEDDING_MODEL = "text-embedding-ada-002"  # OpenAI's best embeddings as of Apr 2023
#BATCH_SIZE = 1000  
#
#law_strings = laws.tolist()
#
#embeddings = []
#for batch_start in range(0, len(law_strings), BATCH_SIZE):
#    batch_end = batch_start + BATCH_SIZE
#    batch = law_strings[batch_start:batch_end]
#    print(f"Batch {batch_start} to {batch_end-1}")
#    response = openai.Embedding.create(model=EMBEDDING_MODEL, input=batch)
#    for i, be in enumerate(response["data"]):
#        assert i == be["index"]  # double check embeddings are in same order as input
#    batch_embeddings = [e["embedding"] for e in response["data"]]
#    embeddings.extend(batch_embeddings)
#
#result = pd.DataFrame({"text": law_strings, "embedding": embeddings})

Batch 0 to 999
Batch 1000 to 1999
Batch 2000 to 2999
Batch 3000 to 3999
Batch 4000 to 4999
Batch 5000 to 5999
Batch 6000 to 6999
Batch 7000 to 7999
Batch 8000 to 8999
Batch 9000 to 9999
Batch 10000 to 10999
Batch 11000 to 11999
Batch 12000 to 12999
Batch 13000 to 13999
Batch 14000 to 14999
Batch 15000 to 15999
Batch 16000 to 16999
Batch 17000 to 17999
Batch 18000 to 18999
Batch 19000 to 19999
Batch 20000 to 20999
Batch 21000 to 21999
Batch 22000 to 22999
Batch 23000 to 23999
Batch 24000 to 24999
Batch 25000 to 25999
Batch 26000 to 26999
Batch 27000 to 27999
Batch 28000 to 28999
Batch 29000 to 29999
Batch 30000 to 30999
Batch 31000 to 31999
Batch 32000 to 32999
Batch 33000 to 33999
Batch 34000 to 34999
Batch 35000 to 35999
Batch 36000 to 36999
Batch 37000 to 37999
Batch 38000 to 38999
Batch 39000 to 39999
Batch 40000 to 40999
Batch 41000 to 41999
Batch 42000 to 42999
Batch 43000 to 43999
Batch 44000 to 44999
Batch 45000 to 45999
Batch 46000 to 46999
Batch 47000 to 47999
Batch 48000 to 4

In [17]:
#df['text'] = df['text'].astype('str')
#df['embedding'] = df['embedding'].astype('str')

In [22]:
def save_index(embeddings: list, index_path: str = 'vector_database.index') -> faiss.IndexFlatL2:
    """ Creates, saves and returns embeddings vectors"""
    # Create a vector index
    dimension = len(embeddings[0])  # Assuming all embeddings have the same dimension
    index = faiss.IndexFlatL2(dimension)  # L2 distance is used for similarity search
    
    # Convert embeddings to numpy array
    embeddings_np = np.array(embeddings).astype('float32')

    # Add embeddings to the index
    index.add(embeddings_np)

    # Save the index
    faiss.write_index(index, index_path)
    
    return index



In [19]:
#loading is super slow
result.to_hdf(r'embeddedfile_all.h5', key='stage', mode='w')

C:\Users\annilo\AppData\Local\Temp\ipykernel_9368\1531630278.py:2: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['text', 'embedding'], dtype='object')]

  result.to_hdf(r'embeddedfile_all.h5', key='stage', mode='w')


In [23]:
index = save_index(embeddings)

## Using the embeddings

In [26]:
def load_index(index_path: str = 'vector_database.index') -> faiss.IndexFlatL2:
    """ Loads index file from disc and returns it as faiss.IndexFlatL2"""
    return faiss.read_index(index_path)

In [27]:
reread = pd.read_hdf('./embeddedfile_all.h5')


In [28]:
index = load_index()

In [29]:
df = reread


The spacial distance cosine is calculated here.

In [42]:
# search function
def strings_ranked_by_relatedness(
    query: str,
    df: pd.DataFrame,
    relatedness_fn=lambda x, y: 1 - spatial_2.distance.cosine(x, y),
    top_n: int = 100
) -> tuple[list[str], list[float]]:
    """Returns a list of strings and relatednesses, sorted from most related to least."""
    query_embedding_response = openai.Embedding.create(
        model=EMBEDDING_MODEL,
        input=query,
    )
    query_embedding = query_embedding_response["data"][0]["embedding"]
    strings_and_relatednesses = [
        (row["text"], relatedness_fn(query_embedding, row["embedding"]))
        for i, row in df.iterrows()
    ]
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    strings, relatednesses = zip(*strings_and_relatednesses)
    return strings[:top_n], relatednesses[:top_n]

In [31]:
# Search function using the vector index and DataFrame
def strings_ranked_by_relatedness_vector(
    query: str,
    index: faiss.IndexFlatL2,
    df: pd.DataFrame,
    relatedness_fn=lambda x, y: 1 - spatial.cosine(x, y),
    top_n: int = 5,
    timeit: bool = False
) -> tuple[list[str], list[float]]:
    """Returns a list of top_n strings and relatednesses, sorted from most related to least."""
    query_embedding_response = openai.Embedding.create(
        model=EMBEDDING_MODEL,
        input=query,
    )
    query_embedding = query_embedding_response["data"][0]["embedding"]

    # Start the timer
    start_time = time.time()

    # Perform similarity search using the vector database
    _, indices = index.search(np.array([query_embedding]), top_n)
    
    strings = df.loc[indices[0], "text"].tolist()
    embeddings = df.loc[indices[0], "embedding"].tolist()
    relatednesses = [relatedness_fn(query_embedding, emb) for emb in embeddings]

    # End the timer
    end_time = time.time()

    if timeit: 
        print(f'Elapsed time: {end_time - start_time} seconds')

    return strings, relatednesses

In [43]:
# examples
strings, relatednesses = strings_ranked_by_relatedness("Lapsendamine", df, top_n=5)
for string, relatedness in zip(strings, relatednesses):
    print(f"{relatedness=:.3f}")
    display(string)


relatedness=0.871


'Seadus TsMS paragrahv 113. Pealkiri: Lapsendamine Sisu: Lapsendamist käsitlev avaldus esitatakse lapsendatava elukoha järgi. Kui lapsendataval ei ole Eestis elukohta, esitatakse avaldus Harju Maakohtusse.'

relatedness=0.871


'Seadus TsMS paragrahv 113. Pealkiri: Lapsendamine Sisu: Lapsendamisasja võib lahendada Eesti kohus, kui lapsendaja, üks lapsendavatest abikaasadest või laps on Eesti Vabariigi kodanik või kui lapsendaja, ühe lapsendava abikaasa või lapse elukoht on Eestis.'

relatedness=0.868


'Seadus TsMS paragrahv 564. Pealkiri: Lapsendamise avaldus Sisu: Avaldaja märgib avalduses oma sünniaasta, -kuu ja -päeva, samuti asjaolud, mis kinnitavad, et ta on suuteline last kasvatama, tema eest hoolitsema ja teda ülal pidama.'

relatedness=0.866


'Seadus PKS paragrahv 158. Pealkiri: Lapsendamise ettevalmistamine Sisu: Kui Sotsiaalkindlustusamet seda nõuab, läbib lapsendada sooviv isik lapsendamisele eelnevalt asjakohase koolitusprogrammi.'

relatedness=0.865


'Seadus PKS paragrahv 147. Pealkiri: Lapsendamise lubatavus Sisu: Lapsendada on lubatud, kui see on lapse huvides vajalik ning on alust arvata, et lapsendaja ja lapse vahel tekib vanema ja lapse suhe. Lapsendajat valides arvestatakse tema isikuomadusi, suhteid lapsendatavaga, varalist seisundit ja võimet täita lapsendamissuhtest tulenevaid kohustusi, samuti võimaluse korral lapse vanemate eeldatavat tahet. Otsustamisel arvestatakse võimaluse korral ka lapse üleskasvatamise järjepidevuse vajadust ning tema rahvuslikku, usulist, kultuurilist ja keelelist päritolu.'

In [37]:
strings, relatednesses = strings_ranked_by_relatedness_vector("Lapsendamine",index,df)
for string, relatedness in zip(strings, relatednesses):
    print(f"{relatedness=:.3f}")
    display(string)

relatedness=0.871


'Seadus TsMS paragrahv 113. Pealkiri: Lapsendamine Sisu: Lapsendamist käsitlev avaldus esitatakse lapsendatava elukoha järgi. Kui lapsendataval ei ole Eestis elukohta, esitatakse avaldus Harju Maakohtusse.'

relatedness=0.871


'Seadus TsMS paragrahv 113. Pealkiri: Lapsendamine Sisu: Lapsendamisasja võib lahendada Eesti kohus, kui lapsendaja, üks lapsendavatest abikaasadest või laps on Eesti Vabariigi kodanik või kui lapsendaja, ühe lapsendava abikaasa või lapse elukoht on Eestis.'

relatedness=0.868


'Seadus TsMS paragrahv 564. Pealkiri: Lapsendamise avaldus Sisu: Avaldaja märgib avalduses oma sünniaasta, -kuu ja -päeva, samuti asjaolud, mis kinnitavad, et ta on suuteline last kasvatama, tema eest hoolitsema ja teda ülal pidama.'

relatedness=0.866


'Seadus PKS paragrahv 158. Pealkiri: Lapsendamise ettevalmistamine Sisu: Kui Sotsiaalkindlustusamet seda nõuab, läbib lapsendada sooviv isik lapsendamisele eelnevalt asjakohase koolitusprogrammi.'

relatedness=0.865


'Seadus PKS paragrahv 147. Pealkiri: Lapsendamise lubatavus Sisu: Lapsendada on lubatud, kui see on lapse huvides vajalik ning on alust arvata, et lapsendaja ja lapse vahel tekib vanema ja lapse suhe. Lapsendajat valides arvestatakse tema isikuomadusi, suhteid lapsendatavaga, varalist seisundit ja võimet täita lapsendamissuhtest tulenevaid kohustusi, samuti võimaluse korral lapse vanemate eeldatavat tahet. Otsustamisel arvestatakse võimaluse korral ka lapse üleskasvatamise järjepidevuse vajadust ning tema rahvuslikku, usulist, kultuurilist ja keelelist päritolu.'

In [46]:
def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """Return the number of tokens in a string."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))


def query_message(
    query: str,
    df: pd.DataFrame,
    model: str,
    token_budget: int
) -> str:
    """Return a message for GPT, with relevant source texts pulled from a dataframe."""
    strings, relatednesses = strings_ranked_by_relatedness_vector(query, index,df)
    introduction = 'Use the following part of the law to answer the subsequent question. Try to find the best answer.' \
    'Formulate the answer including the law name and paragraph number'\
    'The answer should be a coherent sentence. If the answer cannot be found in the laws, write "Ma ei leidnud seadustest vastust"'
    question = f"\n\nQuestion: {query}"
    message = introduction
    for string in strings:
        next_article = f'\n\n"""\n{string}\n"""'
        if (
            num_tokens(message + next_article + question, model=model)
            > token_budget
        ):
            break
        else:
            message += next_article
    return message + question


def ask(
    query: str,
    df: pd.DataFrame = df,
    model: str = GPT_MODEL,
    token_budget: int = 4096 - 500,
    print_message: bool = False,
) -> str:
    """Answers a query using GPT and a dataframe of relevant texts and embeddings."""
    message = query_message(query, df, model=model, token_budget=token_budget)
    if print_message:
        print(message)
    messages = [
        {"role": "system", "content": "Sa vastad Eesti seaduste andmebaasi küsimustele."},
        {"role": "user", "content": message},
    ]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0
    )
    response_message = response["choices"][0]["message"]["content"]
    return response_message

In [47]:
ask("Kuidas astub minister ametisse?")

'Seadus VVS paragrahv 6 kohaselt astub Vabariigi Valitsus või minister ametisse ametivande andmisega Riigikogu ees.'